In [73]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Берем выборку

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
cv = TfidfVectorizer(max_features=100)
cv = CountVectorizer(max_features=100)

X = cv.fit_transform(twenty_train.data).toarray()
target = twenty_train.target

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, 
                                                  random_state=42, stratify=y_train)
print(X_train.shape, X_test.shape, X_val.shape)

(6335, 100) (2263, 100) (2716, 100)


In [75]:
# dataset = load_breast_cancer()
dataset = load_digits()
df = pd.DataFrame(dataset['data'])
target = dataset['target']

# df = pd.read_csv('../data/model_25_25.csv')
# target = df['target'].astype(int).values
# del df['target']

X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape, X_val.shape)

(1005, 64) (360, 64) (432, 64)


# N случайных дихотомий (без отбора)

In [66]:
attempts_data = []
N_attempts = 5
l = np.unique(target).size
N = 100 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    for score_type in wtypes:
        weight_type = None
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))

Training dich classifiers:   0%|          | 0/100 [00:00<?, ?it/s]

Code Matrix shape == (20,100)



Training dich classifiers:   0%|          | 0/100 [00:00<?, ?it/s]


Training dich classifiers:   1%|          | 1/100 [00:00<00:12,  7.69it/s]

Code Matrix shape == (20,100)


...


Training dich classifiers:   1%|          | 1/100 [00:00<00:12,  7.77it/s]

Code Matrix shape == (20,100)



Training dich classifiers:   1%|          | 1/100 [00:00<00:13,  7.52it/s]

Code Matrix shape == (20,100)



100%|██████████| 5/5 [02:56<00:00, 35.35s/it]


In [67]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,5.000000,5.000000,5.000000,5.000000,5.0
mean,0.245214,0.242121,0.136745,0.229971,100.0
std,0.002852,0.007412,0.015048,0.007503,0.0
min,0.241163,0.232695,0.121134,0.220913,100.0
25%,0.244109,0.237113,0.127025,0.224595,100.0
50%,0.245582,0.243004,0.132548,0.229381,100.0
75%,0.246318,0.246318,0.143962,0.236745,100.0
max,0.248895,0.251473,0.159057,0.238218,100.0


# Отбор

In [68]:
attempts_data = []
N_attempts = 5
l = np.unique(target).size
N = 200 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
#     print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    for score_type in wtypes:
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, 
                                      code_matrix, score_type, weight_type=-1)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)

Training dich classifiers:  50%|█████     | 101/200 [00:16<00:15,  6.21it/s]


Training dich classifiers:   0%|          | 0/200 [00:00<?, ?it/s]


Training dich classifiers:  52%|█████▏    | 103/200 [00:16<00:15,  6.10it/s]


Training dich classifiers:   1%|          | 2/200 [00:00<00:27,  7.23it/s]


Training dich classifiers:  52%|█████▎    | 105/200 [00:17<00:15,  6.15it/s]


Training dich classifiers:   2%|▏         | 4/200 [00:00<00:33,  5.88it/s]


Training dich classifiers:  54%|█████▎    | 107/200 [00:17<00:15,  6.14it/s]


Training dich classifiers:   3%|▎         | 6/200 [00:01<00:34,  5.56it/s]


Training dich classifiers:  55%|█████▍    | 109/200 [00:18<00:15,  5.91it/s]


100%|██████████| 5/5 [06:12<00:00, 74.42s/it]


In [69]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.263770,0.260972,0.173196,0.234536,126.600000
std,0.006070,0.007498,0.019270,0.014184,19.501282
min,0.257364,0.252577,0.147275,0.209867,109.000000
25%,0.259941,0.255155,0.161635,0.236377,114.000000
50%,0.261046,0.260677,0.174521,0.238586,125.000000
75%,0.269882,0.265464,0.187776,0.243741,126.000000
max,0.270619,0.270987,0.194772,0.244109,159.000000


# Дихотомии где остались лучшие по критерию

In [70]:
attempts_data = []
N_attempts = 5
l = np.unique(target).size
N = 200 # кол-во дихотомий
num_real_dich = 100 # кол-во дихотомий которые оставляем
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    
    metric_map = np.array([d['accuracy'] for d in dich_classifiers])
    top_dich = np.sort(np.argsort(metric_map)[-num_real_dich:])
    dich_classifiers = [d for i,d in enumerate(dich_classifiers) if i in top_dich]
    code_matrix = code_matrix.T[top_dich].T
    
    for score_type in wtypes:
        weight_type = None
        preds, _ = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))

Training dich classifiers:   0%|          | 1/200 [00:00<00:31,  6.26it/s]

Code Matrix shape == (20,200)



Training dich classifiers:  52%|█████▏    | 104/200 [00:18<00:16,  5.73it/s]


Training dich classifiers:   0%|          | 1/200 [00:00<00:23,  8.33it/s]

Code Matrix shape == (20,200)


...


Training dich classifiers:  52%|█████▏    | 104/200 [00:17<00:16,  5.97it/s]


Training dich classifiers:   0%|          | 1/200 [00:00<00:32,  6.04it/s]

Code Matrix shape == (20,200)



Training dich classifiers:  52%|█████▏    | 104/200 [00:17<00:16,  5.84it/s]


100%|██████████| 5/5 [04:21<00:00, 52.26s/it]


In [71]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,5.000000,5.000000,5.000000,5.000000,5.0
mean,0.236303,0.230928,0.113328,0.212592,100.0
std,0.010979,0.008726,0.029484,0.008285,0.0
min,0.222386,0.219440,0.065169,0.198454,100.0
25%,0.231222,0.227541,0.110457,0.212445,100.0
50%,0.233432,0.230854,0.116716,0.216127,100.0
75%,0.244477,0.233432,0.135862,0.216495,100.0
max,0.250000,0.243373,0.138439,0.219440,100.0


# Локальный метод оптимизации

In [287]:
def score_function(cur_dich):
    # cluster
    target = np.array([cur_dich[i] for i in y_train])
    return -functions.cluster_score(X_train, target, score_type='trace_w')

In [285]:
def score_function(cur_dich):
    # accuracy
    X = X_train
    y = np.array([cur_dich[i] for i in y_train])
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [286]:
def score_function(cur_dich):
    # f1
    X = X_train
    y = np.array([cur_dich[i] for i in y_train])
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = f1_score(y_true, y_pred)
    return accuracy

In [290]:
l = np.unique(target).size
code_matrix = functions.make_random_dichs(l, 1)

Adding dich: 100%|██████████| 1/1 [00:00<00:00, 2830.16it/s]


In [292]:
for i in tqdm(range(20)):
    cur_dich = np.random.randint(0, 2, l)
    while functions.does_dich_exist(cur_dich, code_matrix):
        cur_dich = np.random.randint(0, 2, l)
    new_dich = functions.make_local_optimal_dichotomy(cur_dich.copy(), code_matrix, score_function, verbose=1)
    code_matrix = np.hstack([code_matrix, new_dich.reshape((-1, 1))])

  5%|▌         | 1/20 [00:00<00:02,  8.77it/s]

[1 1 1 0 1 1 1 0 1 1]
[1 1 1 0 1 1 1 0 1 0]
[1 1 0 0 1 1 1 0 1 0]
[1 1 0 0 1 0 1 0 1 0]
[1 1 0 0 1 0 1 0 0 0]
[1 0 0 1 0 0 1 0 0 1]
[1 0 0 1 0 0 0 0 0 1]
[1 0 0 1 0 1 0 0 0 1]
[1 0 1 1 0 1 0 0 0 1]
[1 0 1 1 0 1 0 0 1 1]
[1 1 1 0 0 0 0 1 0 0]


 25%|██▌       | 5/20 [00:00<00:01, 12.42it/s]

[0 1 1 0 0 0 0 1 0 0]
[0 1 1 1 0 0 0 1 0 0]
[0 1 1 1 0 0 0 1 1 0]
[0 1 1 1 0 0 0 1 1 1]
[0 0 0 0 0 1 0 0 0 1]
[1 0 0 0 0 1 0 0 0 1]
[1 0 0 1 0 1 0 0 0 1]
[1 0 1 1 0 1 0 0 0 1]
[1 0 1 1 0 1 0 0 1 1]
[1 1 1 1 0 1 1 0 1 1]
[1 0 0 0 0 1 1 1 0 1]
[1 0 0 0 1 1 1 1 0 1]


 35%|███▌      | 7/20 [00:00<00:01, 12.89it/s]

...

[1 0 0 0 1 1 1 0 0 1]
[1 0 0 0 1 1 1 0 0 0]
[1 1 1 1 1 1 1 1 0 1]
[0 1 0 0 1 1 0 1 1 0]
[0 1 0 0 1 1 1 1 1 0]
[0 1 0 0 1 0 1 1 1 0]
[0 1 0 0 1 0 1 1 0 0]
[0 1 1 0 0 1 0 0 1 0]
[0 1 1 1 0 1 0 0 1 0]
[0 1 1 1 0 1 0 1 1 0]


 90%|█████████ | 18/20 [00:01<00:00, 14.56it/s]

[1 1 0 0 1 1 0 0 0 0]
[1 1 0 0 1 1 1 0 0 0]
[1 0 0 0 1 1 1 0 0 0]
[1 1 0 0 0 1 0 1 1 0]
[0 1 0 0 0 1 0 1 1 0]
[0 1 0 0 1 1 0 1 1 0]
[0 1 0 0 1 1 1 1 1 0]
[0 1 0 0 1 0 1 1 1 0]
[0 1 0 0 1 0 1 1 0 0]
[0 1 1 0 0 1 0 1 1 0]
[0 1 1 1 0 1 0 1 1 0]
[0 1 1 0 0 1 1 1 1 1]


100%|██████████| 20/20 [00:01<00:00, 15.06it/s]

[0 1 1 1 0 1 1 1 1 1]


In [294]:
code_matrix.shape

(10, 21)